### Determining the optimal number of hidden layers and neurons for an Artificial Neural Network (ANN)
This can be challenging and often requires experimentation. However, there are some guidelines and methods that can help you in making an informed decision:
- Start Simple: Begin with a simple architecture and gradually increase complexity if needed
- Grid Search/Random Search: Use grid search or random search to try different architectures
- Cross-Validation: Use cross-validation to evaluate the performance of different architectures
- Heuistics and Rules of Thumb: Some heuristics and empirical rules can provide starting points, such as :
    - The number of neurons in the hidden layer should be between the size of the input layer and the size of output layer
    - A common practice is to start with 1-2 hidden layers

for performing hyperparameter tuning, there is one important function "KerasClassifier" which will help to find out the best model and for this we need to download the library called "scikeras"

In [1]:
import pandas as pd 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

In [8]:
df=pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [9]:
df=df.drop(['RowNumber','CustomerId','Surname'],axis=1)

labelencoder=LabelEncoder()
df['Gender']=labelencoder.fit_transform(df['Gender'])

onehotencoder=OneHotEncoder()
geo=onehotencoder.fit_transform(df[['Geography']]).toarray()
geo_df=pd.DataFrame(geo,columns=onehotencoder.get_feature_names_out(['Geography']))

df=pd.concat([df.drop('Geography',axis=1),geo_df],axis=1)

X=df.drop('Exited', axis=1)
Y=df['Exited']

X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

a function to create the model and try different parameters using KerasClassifier

In [10]:
def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],)))
    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

    return model

In [16]:
# create a Keras Classifier
model=KerasClassifier(layers=1,neurons=32,build_fn=create_model,epochs=50,batch_size=10,verbose=0)

In [17]:
# define the grid search parameters
param_grid={
    'neurons':[16,32,64,128],
    'layers': [1,2],
    # 'batch_size':[10,20],
    'epochs':[50,100]
}

In [ ]:
# perform grid sreach
grid=GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3, verbose=True)
grid_result=grid.fit(X_train,y_train)

print("Best: %f using %s" % (grid_result.best_score_,grid_result.best_params_))